## 1. Loading data from CSV and Excel files
<p>You just got hired as the first and only data practitioner at a small business experiencing exponential growth. The company needs more structured processes, guidelines, and standards. Your first mission is to structure the human resources data. The data is currently scattered across teams and files and comes in various formats: Excel files, CSVs, JSON files, SQL databases…</p>
<p>The Head of People Operations wants to have a general view gathering all available information about a specific employee. Your job is to gather it all in a file that will serve as the reference moving forward. You will merge all of this data in a pandas DataFrame before exporting to CSV.</p>
<p>Data management at your company is not the best, but you need to start somewhere. You decide to tackle the most straightforward tasks first, and to begin by loading the company office addresses. They are currently saved into a CSV file, <code>office_addresses.csv</code>, which the Office Manager sent over to you. Additionally, an HR manager you remember interviewing with gave you access to the Excel file, <code>employee_information.xlsx</code>, where the employee addresses are saved. You need to load these datasets in two separate DataFrames.</p>

In [2]:
# Import the library you need
import pandas as pd

# Load office_addresses.csv
df_office_addresses = pd.read_csv("../input/employee-data/office_addresses.csv")

# Load employee_information.xlsx
df_employee_addresses = pd.read_excel("../input/employee-data/employee_information.xlsx")

# Take a look at the first rows of the DataFrames
display(df_office_addresses)
display(df_employee_addresses)

FileNotFoundError: [Errno 2] No such file or directory: '../input/employee-data/office_addresses.csv'

## 2. Loading employee data from Excel sheets
<p>It turns out the <code>employee_information.xlsx</code> file also holds information about emergency contacts for each employee in a second sheet titled <code>emergency_contacts</code>. However, this sheet was edited at some point, and the header was removed! Looking at the data, you were able to figure out what the header should be, and you confirmed that they were appropriate with the HR manager: <code>employee_id</code>, <code>last_name</code>, <code>first_name</code>, <code>emergency_contact</code>, <code>emergency_contact_number</code>, <code>relationship</code>.</p>

In [ ]:
# Load data from the second sheet of employee_information.xlsx
df_emergency_contacts = pd.read_excel("../input/employee-data/employee_information.xlsx", sheet_name="emergency_contacts", header=None)

# Declare a list of new column names
emergency_contacts_header = ["employee_id", "last_name", "first_name", "emergency_contact", "emergency_contact_number", "relationship"]

# Rename the columns
df_emergency_contacts.columns = emergency_contacts_header

df_emergency_contacts.head()

## 3. Loading role data from JSON files
<p>All right, you're making good progress! Now the next step is to gather information about employee roles, teams, and salaries. This information usually lives in a human resources management system, but the Head of People Operations exported the data for you into a JSON file titled <code>employee_roles.json</code>.</p>
<p>Looking at the JSON file, you see entries are structured in a specific way. It is built as a Python dictionary: the keys are employee IDs, and each employee ID has a corresponding dictionary value holding role, salary, and team information. Here are the first few lines of the file:</p>
<pre><code>{"A2R5H9":
  {
    "title": "CEO",
    "monthly_salary": "$4500",
    "team": "Leadership"
  },
 ...
}
</code></pre>
<p>Load the JSON file to a variable <code>df_employee_roles</code>, choosing the appropriate orientation.</p>

In [ ]:
# Load employee_roles.json
df_employee_roles = pd.read_json("../input/employee-data/employee_roles.json", orient='index')
df_employee_roles = df_employee_roles.reindex(sorted(df_employee_roles.columns), axis=1)

# Take a look at the first rows of the DataFrame
df_employee_roles.head()

## 4. Merging several DataFrames into one
<p>You now have all the data required! All that's left is bringing it all in a unique DataFrame. This unique DataFrame will enable the Head of People Operations to access all employee data at once.</p>
<p>In this step, you will merge all DataFrames. In the next step, you will remove duplicates and reorganize the columns - don't worry about this for now.</p>

In [ ]:
# Merge df_employee_addresses with df_emergency_contacts
df_employees = df_employee_addresses.merge(df_emergency_contacts, how="left", on="employee_id", copy=False)

# Merge df_employees with df_employee_roles
df_employees = df_employees.merge(df_employee_roles, how="left", left_on="employee_id", right_on=df_employee_roles.index, copy=False)

# Merge df_employees with df_office_addresses
df_employees = df_employees.merge(df_office_addresses, how="left", left_on="employee_country", right_on="office_country", copy=False)

# Take a look at the first rows of the DataFrame and its columns
display(df_employees.head())
display(df_employees.columns)

## 5. Editing column names
<p>Now that you merged all of your DataFrames into one let's make sure you have the information required by People Ops.</p>
<p>Currently, your <code>df_employees</code> DataFrame has the following column titles:
<code>employee_id</code>, <code>employee_last_name</code>, <code>employee_first_name</code>, <code>employee_country</code>, <code>employee_city</code>, <code>employee_street</code>, <code>employee_street_number</code>, <code>last_name</code>, <code>first_name</code>, <code>emergency_contact</code>, <code>emergency_contact_number</code>, <code>relationship</code>, <code>monthly_salary</code>, <code>team</code>, <code>title</code>,  <code>office</code>, <code>office_country</code>, <code>office_city</code>, <code>office_street</code>, <code>office_street_number</code>.</p>
<p>The columns <code>employee_last_name</code> and <code>last_name</code> are duplicates. The columns <code>employee_first_name</code> and <code>first_name</code> are duplicates as well. On top of this, People Ops wants to rename some of the columns:</p>
<ul>
<li><code>employee_id</code> should be <code>id</code></li>
<li><code>employee_country</code> should be <code>country</code></li>
<li><code>employee_city</code> should be <code>city</code></li>
<li><code>employee_street</code> should be <code>street</code></li>
<li><code>employee_street_number</code> should be <code>street_number</code></li>
<li><code>emergency_contact_number</code> should be <code>emergency_number</code></li>
<li><code>relationship</code> should be <code>emergency_relationship</code></li>
</ul>
<p><strong>So your header should look like this in the end:</strong>
<code>id</code>, <code>country</code>, <code>city</code>, <code>street</code>, <code>street_number</code>, <code>last_name</code>, <code>first_name</code>, <code>emergency_contact</code>, <code>emergency_number</code>, <code>emergency_relationship</code>, <code>monthly_salary</code>, <code>team</code>, <code>title</code>, <code>office</code>, <code>office_country</code>, <code>office_city</code>, <code>office_street</code>, <code>office_street_number</code>.</p>

In [ ]:
# Drop the columns
df_employees_renamed = df_employees.drop(["employee_first_name", "employee_last_name"], axis=1)

# New columns names
new_column_names = {"employee_id": "id",
                    "employee_country": "country",
                    "employee_city": "city",
                    "employee_street": "street",
                    "employee_street_number": "street_number",
                    "relationship": "emergency_relationship",
                    "emergency_contact_number": "emergency_number"}

# Rename the columns
df_employees_renamed = df_employees_renamed.rename(columns=new_column_names)

# Take a look at the first rows of the DataFrame
df_employees_renamed.head()

## 6. Changing column order
<p>Now that you have the appropriate column names, you can reorder the columns.</p>

In [ ]:
# Declare a list for the new column's order and reorder columns
new_column_order = ["id", "last_name", "first_name", "title", "team", "monthly_salary", 
                    "country", "city", "street", "street_number",
                    "emergency_contact", "emergency_number", "emergency_relationship",
                    "office", "office_country", "office_city", "office_street", "office_street_number"]

# Reorder the columns
df_employees_ordered = df_employees_renamed.reindex(new_column_order, axis=1)

# Take a look at the result
df_employees_ordered.head()

## 7. The last minute request
<p>Last touches! You were ready to let People Ops know that the DataFrame was ready, but the department head just went over to your desk after lunch, asking about some last-minute requirements.</p>
<p>Let's polish the DataFrame before exporting the data, sending it over to People Ops, and deploying the pipeline:</p>
<ul>
<li>All street numbers should be integers</li>
<li>The index should be the actual employee ID rather than the row number</li>
<li>If the value for office is <code>NaN</code> then the employee is remote: add a column named "status", right after <code>monthly_salary</code> indicating whether the employee is "On-site" or "Remote."</li>
</ul>

In [4]:
# Reset the index and drop the column
df_employees_final = df_employees_ordered.set_index("id", drop=True)

status_list = []

# Loop through the row values and append to status_list accordingly
for i, row in df_employees_final.iterrows():
    if pd.isnull(row["office"]):
        status_list.append("Remote")
    else:
        status_list.append("On-site")

# Insert status_list as a new column
df_employees_final.insert(loc=5, column="status", value=status_list)

# Take a look at the first rows of the DataFrame
df_employees_final.head()

NameError: name 'df_employees_ordered' is not defined

## 8. Saving your work
<p>Good job! You now have everything People Ops requested. The different people responsible for these various files can currently keep working on these files if they want. As long as they save it in the <code>datasets</code> folder, People Ops will have to execute this unique script to obtain just one file from the ones scattered across different teams.</p>
<p>You bumped into the Head of People Ops and shared a few caveats and areas of improvement. She booked a meeting with you so you can explain:</p>
<ul>
<li>How the current structure isn't robust to role changes: what if an existing employee takes on a new role?</li>
<li>How the current structure doesn't fit best practices in terms of database schema:<ul>
<li>having data all over the place like it's the case right now is a no-go</li>
<li>but gathering everything in a single table is inefficient: you have to query all information even if all you want is a phone number</li>
<li>there should be a single SQL database for employee data, with several tables that can be joined</li>
<li>views can be built on top of the database to simplify non-data practitioners access.</li></ul></li>
</ul>
<p>In any case, you still need to show up with what was requested - so let's export your DataFrame to a CSV file.</p>

In [ ]:
# Write to CSV
df_employees_final.to_csv("employee_data.csv")